# Use LSTM neural networks 

In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from min_max_scaler import MinMaxScaler

In [74]:
df = pd.read_csv('data/processed.csv')
df.drop(columns=df.columns[0], inplace=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
scaler = MinMaxScaler([-1, 1])
print(device)


cuda


In [59]:
df.iloc[:, 0].to_numpy().reshape((1, -1))

array([[4.46645438, 4.76248682, 5.19528279, ..., 6.2386703 , 6.15265922,
        6.18706366]])

In [65]:
data = torch.FloatTensor(scaler.fit_transform(df.iloc[:, 0].to_numpy()))

In [66]:
data

tensor([-3.0578e-01, -2.5475e-01, -1.8015e-01,  ..., -2.9017e-04,
        -1.5117e-02, -9.1860e-03])

In [67]:
def get_sequences(data, window_lenght):
    sequences = []
    for i in range(len(data)-window_lenght):
        sequences.append([
            data[i:i+window_lenght].to(device),
            data[i+window_lenght:i+window_lenght+1].to(device)
        ])
    return sequences

In [68]:
def split_data(data, spliter_count):
    spliter_count = int(len(data) * spliter_count)
    return data[:spliter_count], data[spliter_count:]

In [69]:
window_lenght = 10
sequences = get_sequences(data, window_lenght)


In [70]:
train, test = split_data(sequences, 0.8)


In [72]:
train

[[tensor([-0.3058, -0.2548, -0.1801, -0.0393,  0.5626,  0.5793,  0.5838,  0.4191,
          -0.9838,  0.3672], device='cuda:0'),
  tensor([0.4412], device='cuda:0')],
 [tensor([-0.2548, -0.1801, -0.0393,  0.5626,  0.5793,  0.5838,  0.4191, -0.9838,
           0.3672,  0.4412], device='cuda:0'),
  tensor([0.3489], device='cuda:0')],
 [tensor([-0.1801, -0.0393,  0.5626,  0.5793,  0.5838,  0.4191, -0.9838,  0.3672,
           0.4412,  0.3489], device='cuda:0'),
  tensor([0.3084], device='cuda:0')],
 [tensor([-0.0393,  0.5626,  0.5793,  0.5838,  0.4191, -0.9838,  0.3672,  0.4412,
           0.3489,  0.3084], device='cuda:0'),
  tensor([0.0252], device='cuda:0')],
 [tensor([ 0.5626,  0.5793,  0.5838,  0.4191, -0.9838,  0.3672,  0.4412,  0.3489,
           0.3084,  0.0252], device='cuda:0'),
  tensor([-0.0429], device='cuda:0')],
 [tensor([ 0.5793,  0.5838,  0.4191, -0.9838,  0.3672,  0.4412,  0.3489,  0.3084,
           0.0252, -0.0429], device='cuda:0'),
  tensor([-0.1757], device='cuda:0'

In [71]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        # input_seq.to(device)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [37]:
model = LSTM()
model.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
epochs = 150

for i in range(epochs):
    for seq, labels in train[:100]:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        
        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu

In [39]:
model(torch.FloatTensor([1,2,3,4,5]).cuda())

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu